In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def extrair_detalhes_holerite(session, url_remun, nome_servidor):
    try:
        response = session.get(url_remun, timeout=15)
        response.encoding = 'iso-8859-1'
        soup = BeautifulSoup(response.text, 'html.parser')
        
        detalhes = {}
        tabela = soup.find('table', class_='detalhe-contracheque')
        
        if not tabela:
            print(f"    ⚠ Tabela não encontrada para: {nome_servidor}")
            return detalhes
        
        secao_atual = ''
        
        for linha in tabela.find_all('tr'):
            classes = linha.get('class', [])
            cols = linha.find_all('td')
            if not cols:
                continue
            
            # ✅ BRUTO e LÍQUIDO ficam na própria linha de seção — extrair o valor corretamente
            if 'bruto' in classes or 'liquido' in classes:
                nome_campo = cols[0].get_text(strip=True)    # "BRUTO" ou "LÍQUIDO"
                valor = cols[1].get_text(strip=True) if len(cols) > 1 else ''
                detalhes[nome_campo] = valor
                # Itens que vêm após o BRUTO são descontos (Previdência, IR...)
                if 'bruto' in classes:
                    secao_atual = 'Descontos'
                continue
            
            # Demais linhas de seção (títulos de grupo de provento)
            if 'secao' in classes:
                secao_atual = cols[0].get_text(strip=True)
                continue
            
            # Linha de dados normal: [campo] [Mês Atual] [Atrasados]
            if len(cols) >= 2:
                chave = cols[0].get_text(strip=True)
                if not chave or chave == '&nbsp;':
                    continue
                
                valor_mes      = cols[1].get_text(strip=True) if len(cols) > 1 else ''
                valor_atrasado = cols[2].get_text(strip=True) if len(cols) > 2 else ''
                
                prefixo = f"{secao_atual} - " if secao_atual else ''
                chave_mes = f"{prefixo}{chave}"
                
                if valor_mes and valor_mes != '-':
                    detalhes[chave_mes] = valor_mes
                
                # Coluna de atrasados só aparece se tiver valor real
                if valor_atrasado and valor_atrasado != '-':
                    detalhes[f"{chave_mes} (Atrasados)"] = valor_atrasado
        
        bruto   = detalhes.get('BRUTO', '?')
        liquido = detalhes.get('LÍQUIDO', '?')
        print(f"    ✔ OK — {len(detalhes)} campos | Bruto: {bruto} | Líquido: {liquido}")
        return detalhes
    
    except Exception as e:
        print(f"    ✖ Erro ao acessar holerite de {nome_servidor}: {e}")
        return {}


def executar_scraping():
    base_url = "https://transparencia.alesc.sc.gov.br/"
    lista_final = []
    total_paginas = 138
    erros_pagina  = 0
    erros_holerite = 0
    
    session = requests.Session()
    session.headers.update({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/120.0.0.0 Safari/537.36'
    })

    print("=" * 60)
    print("INICIANDO SCRAPING — TRANSPARÊNCIA ALESC")
    print(f"Total de páginas a processar: {total_paginas}")
    print("=" * 60)

    for pagina in range(1, total_paginas + 1):
        print(f"\n[Pág {pagina:>3}/{total_paginas}] Buscando lista de servidores...")
        url_lista = f"{base_url}servidores.php?pagina={pagina}&nome=&vinculo=&setor_id=&modalidade="
        
        try:
            res = session.get(url_lista, timeout=15)
            res.encoding = 'iso-8859-1'
            soup = BeautifulSoup(res.text, 'html.parser')
            
            tabela_principal = soup.find('table', class_='pagamentos')
            if not tabela_principal:
                print(f"  ⚠ Tabela 'pagamentos' não encontrada — pulando página {pagina}")
                erros_pagina += 1
                continue
                
            linhas = tabela_principal.find_all('tr')[1:]
            print(f"  → {len(linhas)} servidores encontrados nesta página")

            for idx, linha in enumerate(linhas, start=1):
                cols = linha.find_all('td')
                if len(cols) >= 5:
                    nome   = cols[0].text.strip()
                    vinculo = cols[1].text.strip()
                    info = {
                        'Nome':    nome,
                        'Vínculo': vinculo,
                        'Lotação': cols[2].text.strip(),
                        'Regime':  cols[3].text.strip()
                    }
                    
                    link_tag = cols[4].find('a')
                    if link_tag:
                        href = link_tag['href']
                        url_holerite = f"{base_url}{href}"
                        print(f"  [{idx:>2}/{len(linhas)}] {nome[:40]:<40} | {vinculo[:20]}")
                        print(f"         URL: {url_holerite}")
                        dados_remuneracao = extrair_detalhes_holerite(session, url_holerite, nome)
                        if not dados_remuneracao:
                            erros_holerite += 1
                        info.update(dados_remuneracao)
                    else:
                        print(f"  [{idx:>2}/{len(linhas)}] {nome[:40]:<40} | sem link de remuneração")
                    
                    lista_final.append(info)
            
            print(f"  ✔ Página {pagina} concluída — total acumulado: {len(lista_final)} servidores")
            time.sleep(0.3)
            
        except Exception as e:
            print(f"  ✖ Erro na página {pagina}: {e}")
            erros_pagina += 1

    print("\n" + "=" * 60)
    print("EXPORTANDO DADOS...")
    
    df = pd.DataFrame(lista_final)
    
    # ✅ Ordenação de colunas por seção lógica
    # Seção 1 — Identificação
    cols_id = ['Nome', 'Vínculo', 'Lotação', 'Regime']
    # Seção 2 — Totalizadores (mais importantes, logo no início)
    cols_totais = ['BRUTO', 'LÍQUIDO']
    # Seção 3 — Rendimentos (todos os campos prefixados com "Rendimentos permanentes - ")
    cols_rendimentos = sorted([c for c in df.columns if c.startswith('Rendimentos permanentes')])
    # Seção 4 — Outros proventos
    cols_outros = sorted([c for c in df.columns if c.startswith('Outros')])
    # Seção 5 — Descontos (Previdência, IR etc.)
    cols_descontos = sorted([c for c in df.columns if c.startswith('Descontos')])
    # Demais colunas não categorizadas
    todas_mapeadas = set(cols_id + cols_totais + cols_rendimentos + cols_outros + cols_descontos)
    cols_resto = [c for c in df.columns if c not in todas_mapeadas]
    
    cols_ordenadas = (
        [c for c in cols_id     if c in df.columns] +
        [c for c in cols_totais if c in df.columns] +
        cols_rendimentos +
        cols_outros +
        cols_descontos +
        cols_resto
    )
    df = df[cols_ordenadas]
    
    df.to_csv('relatorio_completo_alesc.csv', index=False, encoding='utf-8-sig')
    
    print("=" * 60)
    print(f"✔ Arquivo 'relatorio_completo_alesc.csv' criado com sucesso!")
    print(f"  Total de servidores exportados : {len(lista_final)}")
    print(f"  Total de colunas               : {len(df.columns)}")
    print(f"  Erros em páginas               : {erros_pagina}")
    print(f"  Holerites sem dados            : {erros_holerite}")
    print("=" * 60)

executar_scraping()

INICIANDO SCRAPING — TRANSPARÊNCIA ALESC
Total de páginas a processar: 138

[Pág   1/138] Buscando lista de servidores...
  → 20 servidores encontrados nesta página
  [ 1/20] ABEL VEIGA                               | Comissionado
         URL: https://transparencia.alesc.sc.gov.br/remuneracao.php?ID=G18P
    ✔ OK — 7 campos | Bruto: 5.652,53 | Líquido: 4.806,96
  [ 2/20] ABRAHAO MUSSI FILHO                      | Comissionado
         URL: https://transparencia.alesc.sc.gov.br/remuneracao.php?ID=od1x
    ✔ OK — 6 campos | Bruto: 1.621,33 | Líquido: 1.451,10
  [ 3/20] ACACIO JACQUES                           | Aposentado
         URL: https://transparencia.alesc.sc.gov.br/remuneracao.php?ID=OzA
    ✔ OK — 7 campos | Bruto: 23.778,20 | Líquido: 21.635,84
  [ 4/20] ACACIO MANOEL DA SILVA                   | Militar
         URL: https://transparencia.alesc.sc.gov.br/remuneracao.php?ID=QRkx
    ⚠ Tabela não encontrada para: ACACIO MANOEL DA SILVA
  [ 5/20] ACINDINO GRACILIANO DE QUADROS  